In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf 
import keras

In [2]:
# # Place tensors on the CPU
# with tf.device('/CPU:0'):
#     a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
#     b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])


In [3]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [4]:
# import os
# import tensorflow as tf

# os.environ['CUDA_VISIBLE_DEVICES'] = ''

# if tf.test.gpu_device_name():
#     print('GPU found')
# else:
#     print("No GPU found")

In [5]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]='-1' 

In [6]:
# with tf.compat.v1.Session() as sess:
#     devices = sess.list_devices()

In [7]:
# gpu_devices = tf.config.experimental.list_physical_devices('CPU')
# for device in gpu_devices: 
#     tf.config.experimental.set_memory_growth(device, True)
#     print(device)

In [8]:
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [9]:
# config = tf.compat.v1.ConfigProto( device_count = {'CPU': 1} ) 
# sess = tf.compat.v1.Session(config=config) 
# keras.backend.set_session(sess)

In [10]:
# print('GPU:',tf.test.is_gpu_available())
# print('CPU:',tf.test.is_cpu_available())

In [11]:
# tf.config.list_physical_devices('GPU')

In [12]:
# tf.config.list_physical_devices('CPU')

In [13]:
#顯示所有列
pd.set_option('display.max_columns', None)

In [14]:
# Reading the data
train = pd.read_csv('tbrain_cc_training_48tags_hash_final.csv')
print(train.shape)
print(train.info())
train.head(5)

(32975653, 53)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32975653 entries, 0 to 32975652
Data columns (total 53 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   dt                        int64  
 1   chid                      int64  
 2   shop_tag                  object 
 3   txn_cnt                   int64  
 4   txn_amt                   float64
 5   domestic_offline_cnt      int64  
 6   domestic_online_cnt       int64  
 7   overseas_offline_cnt      int64  
 8   overseas_online_cnt       int64  
 9   domestic_offline_amt_pct  float64
 10  domestic_online_amt_pct   float64
 11  overseas_offline_amt_pct  float64
 12  overseas_online_amt_pct   float64
 13  card_1_txn_cnt            int64  
 14  card_2_txn_cnt            int64  
 15  card_3_txn_cnt            int64  
 16  card_4_txn_cnt            int64  
 17  card_5_txn_cnt            int64  
 18  card_6_txn_cnt            int64  
 19  card_7_txn_cnt            int64  
 20  card_8_

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_cnt,card_2_txn_cnt,card_3_txn_cnt,card_4_txn_cnt,card_5_txn_cnt,card_6_txn_cnt,card_7_txn_cnt,card_8_txn_cnt,card_9_txn_cnt,card_10_txn_cnt,card_11_txn_cnt,card_12_txn_cnt,card_13_txn_cnt,card_14_txn_cnt,card_other_txn_cnt,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1,10321418,45,3,3891.965283,3,0,0,0,1.00,0.00,0.0,0.0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1
1,1,10414574,15,2,10616.561549,2,0,0,0,1.00,0.00,0.0,0.0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1
2,1,10134567,48,2,23527.655416,0,2,0,0,0.00,1.00,0.0,0.0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1
3,1,10001003,48,9,17751.558260,7,2,0,0,0.75,0.25,0.0,0.0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1
4,1,10267183,2,1,21701.307598,1,0,0,0,1.00,0.00,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0


In [15]:
predict = pd.DataFrame(pd.read_csv('需預測的顧客名單及提交檔案範例.csv'))
predict = predict.drop(columns = ['top1','top2','top3'])
print(predict.shape)
print(predict.info())
predict.head(5)

(500000, 1)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   chid    500000 non-null  int64
dtypes: int64(1)
memory usage: 3.8 MB
None


,chid
0,10128239
1,10077943
2,10277876
3,10364842
4,10392717


In [16]:
train.dtypes

dt                            int64
chid                          int64
shop_tag                     object
txn_cnt                       int64
txn_amt                     float64
domestic_offline_cnt          int64
domestic_online_cnt           int64
overseas_offline_cnt          int64
overseas_online_cnt           int64
domestic_offline_amt_pct    float64
domestic_online_amt_pct     float64
overseas_offline_amt_pct    float64
overseas_online_amt_pct     float64
card_1_txn_cnt                int64
card_2_txn_cnt                int64
card_3_txn_cnt                int64
card_4_txn_cnt                int64
card_5_txn_cnt                int64
card_6_txn_cnt                int64
card_7_txn_cnt                int64
card_8_txn_cnt                int64
card_9_txn_cnt                int64
card_10_txn_cnt               int64
card_11_txn_cnt               int64
card_12_txn_cnt               int64
card_13_txn_cnt               int64
card_14_txn_cnt               int64
card_other_txn_cnt          

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32975653 entries, 0 to 32975652
Data columns (total 53 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   dt                        int64  
 1   chid                      int64  
 2   shop_tag                  object 
 3   txn_cnt                   int64  
 4   txn_amt                   float64
 5   domestic_offline_cnt      int64  
 6   domestic_online_cnt       int64  
 7   overseas_offline_cnt      int64  
 8   overseas_online_cnt       int64  
 9   domestic_offline_amt_pct  float64
 10  domestic_online_amt_pct   float64
 11  overseas_offline_amt_pct  float64
 12  overseas_online_amt_pct   float64
 13  card_1_txn_cnt            int64  
 14  card_2_txn_cnt            int64  
 15  card_3_txn_cnt            int64  
 16  card_4_txn_cnt            int64  
 17  card_5_txn_cnt            int64  
 18  card_6_txn_cnt            int64  
 19  card_7_txn_cnt            int64  
 20  card_8_txn_cnt        

In [18]:
# train空值往前填充
train.fillna(method='ffill', inplace=True)

In [19]:
# 把shop_tag的other取代為49
train['shop_tag']= train['shop_tag'].replace('other','0')

In [20]:
# 把shop_tag的object型態轉換為數字
train["shop_tag"] = train["shop_tag"].astype(str).astype(int)

In [21]:
train.dtypes

dt                            int64
chid                          int64
shop_tag                      int64
txn_cnt                       int64
txn_amt                     float64
domestic_offline_cnt          int64
domestic_online_cnt           int64
overseas_offline_cnt          int64
overseas_online_cnt           int64
domestic_offline_amt_pct    float64
domestic_online_amt_pct     float64
overseas_offline_amt_pct    float64
overseas_online_amt_pct     float64
card_1_txn_cnt                int64
card_2_txn_cnt                int64
card_3_txn_cnt                int64
card_4_txn_cnt                int64
card_5_txn_cnt                int64
card_6_txn_cnt                int64
card_7_txn_cnt                int64
card_8_txn_cnt                int64
card_9_txn_cnt                int64
card_10_txn_cnt               int64
card_11_txn_cnt               int64
card_12_txn_cnt               int64
card_13_txn_cnt               int64
card_14_txn_cnt               int64
card_other_txn_cnt          

In [22]:
# 對txn_amt、slam做Normalization，因為小數位數過大會報錯
train['txn_amt'] = (train['txn_amt'] - train['txn_amt'].min())/\
                            (train['txn_amt'].max() - train['txn_amt'].min())
train['slam'] = (train['slam'] - train['slam'].min())/\
                            (train['slam'].max() - train['slam'].min())

In [23]:
# 對卡片消費次數做加總，並刪掉個別消費次數，這邊可能要再試
train["sum_card_txn_cnt"] = train["card_1_txn_cnt"] + train["card_2_txn_cnt"] + train["card_3_txn_cnt"] + train["card_4_txn_cnt"] + train["card_5_txn_cnt"] + train["card_6_txn_cnt"] + train["card_7_txn_cnt"] + train["card_8_txn_cnt"] + train["card_9_txn_cnt"] + train["card_10_txn_cnt"] + train["card_11_txn_cnt"] + train["card_12_txn_cnt"] + train["card_13_txn_cnt"] + train["card_14_txn_cnt"] + train["card_other_txn_cnt"]
train = train.drop(columns = ['card_1_txn_cnt','card_2_txn_cnt','card_3_txn_cnt','card_4_txn_cnt','card_5_txn_cnt','card_6_txn_cnt','card_7_txn_cnt','card_8_txn_cnt','card_9_txn_cnt','card_10_txn_cnt','card_11_txn_cnt','card_12_txn_cnt','card_13_txn_cnt','card_14_txn_cnt','card_other_txn_cnt'])

# # 對卡片消費次數做加總，不刪掉個別次數，這邊可能要再試
# train["sum_card_txn_cnt"] = train["card_1_txn_cnt"] + train["card_2_txn_cnt"] + train["card_3_txn_cnt"] + train["card_4_txn_cnt"] + train["card_5_txn_cnt"] + train["card_6_txn_cnt"] + train["card_7_txn_cnt"] + train["card_8_txn_cnt"] + train["card_9_txn_cnt"] + train["card_10_txn_cnt"] + train["card_11_txn_cnt"] + train["card_12_txn_cnt"] + train["card_13_txn_cnt"] + train["card_14_txn_cnt"] + train["card_other_txn_cnt"]

In [24]:
# 看shop_tag分布有無平均，這邊看起來是沒平均，不過先不做處理，之後再看
train.shop_tag.value_counts()

37    4092401
0     2988218
15    2573145
36    2550500
45    2411310
10    2234101
2     1971956
48    1806496
12    1130825
19    1022234
25     795763
6      634849
18     629407
16     628987
13     625801
31     607888
22     547293
39     535248
21     503873
26     476687
46     424055
9      349932
17     345814
11     280455
33     261088
5      231969
47     225170
32     202863
35     185585
34     183470
41     166662
7      160780
20     130038
8      129061
23     124889
40     118913
43     114998
38     113608
3      109286
42      91492
14      73457
30      62032
44      55273
24      31530
1       20800
4        5997
27       4754
28       4585
29        115
Name: shop_tag, dtype: int64

In [25]:
# predict_test是之後要預測並提交上傳的資料，在這邊先拆分出來 抓倒數幾個月中消費次數最多的時候根據chid做刪除重複項的動作
predict_test = train.iloc[train[(train['dt']==24) | (train['dt']==23) | (train['dt']==22) | (train['dt']==21) | (train['dt']==20) | (train['dt']==19) | (train['dt']==18) | (train['dt']==17) | (train['dt']==16) | (train['dt']==15) | (train['dt']==14) | (train['dt']==13)].groupby(['chid']).apply(lambda x: x['txn_cnt'].idxmax())]

In [26]:
predict_test.head(5)

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card,sum_card_txn_cnt
16536056,14,10000000,10,4,1.873943e-111,0,4,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1,4
21082844,17,10000001,37,6,1.428726e-111,6,0,0,0,1.0,0.0,0.0,0.0,0.8,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,11.0,1.0,3.0,30.0,8.670854e-214,0.0,5.0,1,6
14936013,13,10000002,0,1,1.605622e-111,0,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,13.0,1.0,99.0,30.0,1.129543e-213,0.0,3.0,1,1
27393689,21,10000003,37,2,1.205646e-111,2,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,6.0,15.0,1.0,99.0,30.0,1.129543e-213,0.0,6.0,1,2
20388560,17,10000004,0,2,1.106555e-111,1,0,0,1,0.6,0.0,0.0,0.4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,15.0,1.0,99.0,30.0,6.369410e-214,1.0,2.0,1,2


In [27]:
len(predict_test)

500000

In [28]:
# 按照要提交的檔案排序
predict_test = predict_test.set_index('chid', drop= False)
predict_test = predict_test.loc[predict.chid]
predict_test.head(5)

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card,sum_card_txn_cnt
chid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10128239,18,10128239,0,2,1.086396e-111,2,0,0,0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1,2
10077943,13,10077943,13,16,1.888839e-111,0,0,0,16,0.00,0.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,12.0,1.0,2.0,30.0,7.594606e-214,0.0,5.0,1,16
10277876,15,10277876,37,7,1.904996e-111,4,3,0,0,0.51,0.49,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,6.0,1.0,1.0,30.0,6.083676e-214,1.0,2.0,1,7
10364842,14,10364842,37,7,2.523124e-111,7,0,0,0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,7.0,1.0,99.0,30.0,1.048096e-213,1.0,4.0,1,7
10392717,13,10392717,48,9,2.320924e-111,9,0,0,0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,5.0,1.0,2.0,30.0,3.248696e-214,1.0,2.0,1,9


In [29]:
train.columns

Index(['dt', 'chid', 'shop_tag', 'txn_cnt', 'txn_amt', 'domestic_offline_cnt',
       'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
       'domestic_offline_amt_pct', 'domestic_online_amt_pct',
       'overseas_offline_amt_pct', 'overseas_online_amt_pct',
       'card_1_txn_amt_pct', 'card_2_txn_amt_pct', 'card_3_txn_amt_pct',
       'card_4_txn_amt_pct', 'card_5_txn_amt_pct', 'card_6_txn_amt_pct',
       'card_7_txn_amt_pct', 'card_8_txn_amt_pct', 'card_9_txn_amt_pct',
       'card_10_txn_amt_pct', 'card_11_txn_amt_pct', 'card_12_txn_amt_pct',
       'card_13_txn_amt_pct', 'card_14_txn_amt_pct', 'card_other_txn_amt_pct',
       'masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam',
       'gender_code', 'age', 'primary_card', 'sum_card_txn_cnt'],
      dtype='object')

In [30]:
train.head(5)

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card,sum_card_txn_cnt
0,1,10321418,45,3,5.830816e-112,3,0,0,0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,5.0,1.0,99.0,30.0,1.394080e-213,1.0,4.0,1,3
1,1,10414574,15,2,1.596363e-111,2,0,0,0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,15.0,1.0,2.0,30.0,1.944332e-213,1.0,3.0,1,2
2,1,10134567,48,2,3.541844e-111,0,2,0,0,0.00,1.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,11.0,1.0,3.0,30.0,1.648096e-213,0.0,4.0,1,2
3,1,10001003,48,9,2.671485e-111,7,2,0,0,0.75,0.25,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,11.0,1.0,2.0,30.0,8.190756e-214,0.0,3.0,1,9
4,1,10267183,2,1,3.266644e-111,1,0,0,0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,15.0,1.0,99.0,30.0,8.190756e-214,0.0,6.0,0,1


In [31]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, test_size=0.2, random_state=42)

In [32]:
target_column_train = ['shop_tag']
predictors_train = list(set(list(train.columns))-set(target_column_train))

X_train = train[predictors_train].values
y_train = train[target_column_train].values

print(X_train.shape)
print(y_train.shape)

(26380522, 38)
(26380522, 1)


In [33]:
target_column_test = ['shop_tag'] 
predictors_test = list(set(list(test.columns))-set(target_column_test))

X_test = test[predictors_test].values
y_test = test[target_column_test].values

print(X_test.shape)
print(y_test.shape)

(6595131, 38)
(6595131, 1)


In [34]:
len(X_train)

26380522

In [35]:
len(y_train)

26380522

In [36]:
X_train

array([[3.04808053e-111, 4.00000000e+000, 4.00000000e+000, ...,
        0.00000000e+000, 1.00000000e+000, 0.00000000e+000],
       [3.04908253e-111, 3.00000000e+000, 1.00000000e+001, ...,
        0.00000000e+000, 1.00000000e+000, 0.00000000e+000],
       [1.44732208e-111, 4.00000000e+000, 1.80000000e+001, ...,
        0.00000000e+000, 1.00000000e+000, 0.00000000e+000],
       ...,
       [1.98369844e-111, 5.00000000e+000, 1.80000000e+001, ...,
        1.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [8.66526765e-112, 5.00000000e+000, 2.40000000e+001, ...,
        0.00000000e+000, 1.00000000e+000, 0.00000000e+000],
       [1.19953266e-111, 6.00000000e+000, 2.40000000e+001, ...,
        0.00000000e+000, 1.00000000e+000, 0.00000000e+000]])

In [37]:
y_train

array([[16],
       [45],
       [48],
       ...,
       [30],
       [48],
       [37]])

In [38]:
# # 建立及訓練 LSTM 模型
# # model = Sequential()
# # model.add(LSTM(4, input_shape=(1, look_back)))
# # model.add(Dense(1))
# # model.compile(loss='mean_squared_error', optimizer='adam')
# # model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=2)

# model = Sequential()
# model.add(LSTM(25, dropout=0.2, input_shape=(X_train.shape[1],1)))
# model.add(Dense(1, activation='softmax'))
# # model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# # model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=2)

# # model.fit(X_train[0:3000], y_train[0:3000], batch_size=1, epochs=5)

# model.fit(X_train[0:3000], y_train[0:3000], epochs=5, batch_size=1, verbose=2)

In [39]:
np.unique(y_train)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48])

In [40]:
len(np.unique(y_train))

49

In [41]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(49, activation='softmax'))
 
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2021-11-11 01:26:34.311150: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-11 01:26:34.327811: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-11 01:26:34.505156: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-11 01:26:34.506229: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-11 01:26:34.507511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

In [ ]:
with tf.device('/cpu:0'):
    model.fit(X_train, y_train, epochs=1, batch_size=1, verbose=2)

2021-11-11 01:26:51.561337: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4009839344 exceeds 10% of free system memory.


In [ ]:
# print(X_train[0:300].shape)
# print(predict_test.shape)

In [ ]:
# 預測
# trainPredict = model.predict(X_train)
# testPredict = model.predict(y_train)

# 模型預測
with tf.device('/cpu:0'):
    predictions = model.predict(predict_test.drop(columns = ['shop_tag'], axis=1, inplace=False))

In [ ]:
len(predictions[0])

In [ ]:
predictions[0]

In [ ]:
# 按降序獲取前 n 個結果
best_n = np.argsort(-predictions, axis=1)[:, :49]
best_n

In [ ]:
# 建立best_predictions_final來儲存前三高的預測值
best_predictions_final = np.zeros((len(best_n), 3), dtype=int)

In [ ]:
# 過濾16個前三高的標籤出來
for a in range(len(best_n)):
    sum = 0
    for i in range(49): 
        if (best_n[a, i] == [2,6,10,12,13,15,18,19,21,22,25,26,39,37,39,48]).any():
            best_predictions_final[a, sum] = int(best_n[a, i])
            sum += 1 
        if sum == 3: 
            break

In [ ]:
submission = pd.DataFrame({ 'chid':predict.chid,'top1':best_predictions_final[0:len(best_predictions_final),0] , 'top2':best_predictions_final[0:len(best_predictions_final),1] , 'top3':best_predictions_final[0:len(best_predictions_final),2] })

In [ ]:
filename = 'submission.csv'
submission.to_csv(filename,index=False)
print(submission)
print('Saved file: ' + filename)

In [ ]:
# 保存模型
model.save('LSTM_model_allData.h5') 